## hetrec2011-lastfm-2k EDA

In [ ]:
import pandas as pd
from ydata_profiling import ProfileReport
from pathlib import Path

### user_artists.dat
This file contains the artists listened by each user.
        
It also provides a listening count for each [user, artist] pair

userID \t artistID \t weight
2	51	13883

In [ ]:
user_artists = pd.read_csv(Path() / "../../data/raw/hetrec2011-lastfm-2k/user_artists.dat", sep="\t")

In [ ]:
user_artists_report = ProfileReport(user_artists, title="User-artists dataset report")
user_artists_report.to_file("../../reports/user_artists_profile_report.html")

### user_friends.dat
These files contain the friend relations between users in the database.

userID \t friendID
2	275

In [ ]:
user_friends = pd.read_csv(Path() / "../../data/raw/hetrec2011-lastfm-2k/user_friends.dat", sep="\t")

In [ ]:
user_friends_report = ProfileReport(user_friends, title="User-friends dataset report")
user_friends_report.to_file("../../reports/user_friends_profile_report.html")

### user_taggedartists-timestamps.dat
These files contain the tag assignments of artists provided by each particular user.
        
They also contain the timestamps when the tag assignments were done.

user_taggedartists-timestamps.dat

userID \t artistID \t tagID \t timestamp
2	52	13	1238536800000

In [ ]:
user_taggedartists = pd.read_csv(Path() / "../../data/raw/hetrec2011-lastfm-2k/user_taggedartists-timestamps.dat", sep="\t")

In [ ]:
user_taggedartists_report = ProfileReport(user_taggedartists, title="User tagged-artists dataset report")
user_taggedartists_report.to_file("../../reports/user_taggedartists_profile_report.html")

### tags.dat
This file contains the set of tags available in the dataset.

tagID \t tagValue
1	metal

In [ ]:
tags = pd.read_csv(Path() / "../../data/raw/hetrec2011-lastfm-2k/tags.dat", sep="\t")

In [ ]:
tags_report = ProfileReport(tags, title="Tags dataset report")
tags_report.to_file("../../reports/tags_profile_report.html")

### artists.dat
This file contains information about music artists listened and tagged by the users.

id \t name \t url \t pictureURL

Example:
707	Metallica	http://www.last.fm/music/Metallica	http://userserve-ak.last.fm/serve/252/7560709.jpg

In [ ]:
artists = pd.read_csv(Path() / "../../data/raw/hetrec2011-lastfm-2k/artists.dat", sep="\t")

In [ ]:
artists_report = ProfileReport(artists, title="Artists dataset report")
artists_report.to_file("../../reports/artists_profile_report.html")